In [142]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import re
from unicodedata import normalize
import time
import datetime

import json
from pandas.io.json import json_normalize

#Import for similarity of strings
import string

import geopandas as gpd

pd.options.mode.chained_assignment = None

In [30]:
# ====== Connection ======
# Connecting to PostgreSQL by providing a sqlachemy engine
# Local
username= 'ds4a_final'
password='qwerty.123'
host = 'localhost'
port='5432'
database = 'ds4a_finalproject'
# Remote
# username= 'final_project'
# password='*******'
# host = '*****'
# port='5432'
# database = 'ds4a_final'

engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}',echo=False)

In [3]:
os.chdir(os.path.join('..')) #Run only once

In [7]:
basepath = os.getcwd()
ds_path = os.path.join(basepath, 'datasets\\final\\')
pj_path = os.path.join(basepath, 'final\\')

### NEIGHBORHOOD - MASTER DATA

In [33]:
ds_barrios_poligono = os.path.join(ds_path, 'raw\\barriosbquilla.geojson') 
df_barrios = gpd.read_file(ds_barrios_poligono)
df_barrios.columns = map(str.lower, df_barrios.columns)

#Deleting unnecessary columns
del df_barrios['name']
del df_barrios['description']
del df_barrios['id']
del df_barrios['shape__are']
del df_barrios['shape__len']
del df_barrios['geometry']

df_barrios = df_barrios[df_barrios['nombre'] != '']
df_barrios['barrio_id'] = df_barrios.index
df_barrios.to_sql(name='barrio', con=engine, if_exists = 'append', index=False)

### CRIME DATASET

In [39]:
def replaceValues(text):
    dic = { '2010.0': 'Q',
            '2011.0': 'Q',
            '2012.0': 'Q',
            'ARMA EMPLEADA': 'ARMAEMPLEADA',
            'BARRIO': 'BARRIO',
            'CANTIDAD': 'CANTIDAD',
            'CLASE': 'CLASE',
            'CLASE DE EMPLEADO': 'CLASEEMPLEADO',
            'CLASE DE SITIO': 'CLASESITIO',
            'CLASE EMPLEADO': 'CLASEEMPLEADO',
            'CLASE EMPRESA': 'CLASEEMPRESA',
            'CLASE SITIO': 'CLASESITIO',
            'CODIGO DANE': 'CODIGODANE',
            'COLOR': 'COLOR',
            'CRIMEN': 'CRIMEN',
            'DELITO': 'DELITO',
            'DEPARTAMENTO': 'DEPARTAMENTO',
            'DESCRIPCION CONDUCTA': 'DELITO',
            'DIA': 'DIA',
            'EDAD': 'EDAD',
            'ESCOLARIDAD': 'ESCOLARIDAD',
            'ESTADO CIVIL': 'ESTADOCIVIL',
            'FECHA': 'FECHA',
            'HORA': 'HORA',
            'LINEA': 'LINEA',
            'MARCA': 'MARCA',
            'MODELO': 'MODELO',
            'MOVIL AGRESOR': 'MOVILAGRESOR',
            'MOVIL VICTIMA': 'MOVILVICTIMA',
            'MUNICIPIO': 'MUNICIPIO',
            'PAIS DE NACIMIENTO': 'PAISNACIMIENTO',
            'PAIS NACE': 'PAISNACIMIENTO',
            'PROFESION': 'PROFESION',
            'PROFESIONES': 'PROFESION',
            'SEXO': 'SEXO',
            'ZONA': 'ZONA'}
    
    # -> NFD y eliminar diacríticos
    text = re.sub(
            r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
            normalize( 'NFD', text), 0, re.I
        )

    # -> NFC
    text = normalize( 'NFC', text ).upper().strip()
    
    #Returns the value of the dictionary in lower case
    return dic[text].lower()

In [37]:
ds_police_path = os.path.join(ds_path, 'raw\\National Police\\')

filenames = []
filenames = [entry for entry in os.listdir(ds_police_path) if os.path.isfile(os.path.join(ds_police_path, entry))]

In [41]:
df_crime = pd.DataFrame()
for file in filenames:
    raw_data = pd.read_excel(os.path.join(ds_police_path,file), header=None)
    #print(file)
    # looking for the header row
    for i, row in raw_data.iterrows():
        if row.notnull().all():
            data = raw_data.iloc[(i+1):].reset_index(drop=True)
            data.columns = list(raw_data.iloc[i].apply(lambda s: replaceValues(str(s)) ))
            data['crimen'] = file[ 0 : file.rfind('-') ].upper()

            data = data[ ( data['departamento'].str.strip() == 'ATLÁNTICO' ) & 
                         ( data['municipio'].str.strip() == 'BARRANQUILLA (CT)') ]
            df_crime = pd.concat([df_crime, data], sort=True)
            break
df_crime.to_csv(os.path.join(ds_path, 'raw\\crime_all.csv') , header=True, index=False, sep=',') 

In [ ]:
dane_ds = os.path.join(basepath, 'Datasets/dane.csv')
barrios_estrato_ds = os.path.join(basepath, 'Datasets/barrios_por_estrato.csv')

In [43]:
df_tmp = df_crime.copy()

#### DATA WRANGLING

In [67]:
#Para los siguientes crimenes no aplica la columna SEXO: HURTO A ENTIDADES FINANCIERAS, HURTO A ENTIDADES COMERCIALES
#HURTO COMERCIO, TERRORISMO
df_crime['sexo'].fillna('NO APLICA', inplace=True)
df_crime['sexo'] = df_crime['sexo'].apply(lambda data: 'NO REPORTA' if (data == '-') | (data == 'NO REPORTADA') | (data == 'NO REPORTADO') else data )

In [68]:
df_crime['color'].fillna('NO APLICA', inplace=True)
df_crime['color'] = df_crime['color'].apply(lambda data: 'NO REPORTADO' if data == '-' else data )

In [69]:
df_crime['linea'].fillna('NO APLICA', inplace=True)
df_crime['linea'] = df_crime['linea'].apply(lambda data: 'NO REPORTADO' if data == '-' else data )

In [70]:
df_crime['modelo'].fillna('NO APLICA', inplace=True)
df_crime['modelo'] = df_crime['modelo'].apply(lambda data: 'NO REPORTADO' if data == '-' else data )
df_crime['modelo'] = df_crime['modelo'].apply(lambda data: 'NO REPORTADO' if data == 'X' else data )

In [71]:
df_crime['marca'].fillna('NO APLICA', inplace=True)
df_crime['marca'] = df_crime['marca'].apply(lambda data: 'NO REPORTADO' if data == '-' else data )

In [72]:
df_crime['armaempleada'] = df_crime['armaempleada'].apply(lambda data: 'NO REPORTADO' if data == '-' else data )

In [73]:
df_crime['barrio'].fillna('NO REPORTADO', inplace=True)
df_crime['barrio'] = df_crime['barrio'].apply(lambda data: 'NO REPORTADO' if data == '-' else data )

In [74]:
df_crime['cantidad'].fillna(0, inplace=True)

In [75]:
df_crime['clase'].fillna('NO APLICA', inplace=True) #Solo aplica a vehiculos o motos

In [76]:
#'HURTO COMERCIO', 'HURTO A ENTIDADES COMERCIALES', 'HURTO A ENTIDADES FINANCIERAS', 'TERRORISMO'
df_crime['claseempleado'].fillna('NO APLICA', inplace=True) 
df_crime['claseempleado'] = df_crime['claseempleado'].apply(lambda data: 'NO REPORTADO' if data == '-' else data )

In [77]:
#Solo aplica a 'HURTO A ENTIDADES COMERCIALES'
df_crime['claseempresa'].fillna('NO APLICA', inplace=True) 

In [78]:
#Solo aplica a 'DELITOS SEXUALES', 'TERRORISMO'
df_crime['delito'].fillna('NO APLICA', inplace=True) 

In [80]:
# 'HURTO COMERCIO', 'HURTO A ENTIDADES COMERCIALES', 'HURTO A ENTIDADES FINANCIERAS', 'TERRORISMO'
df_crime['edad'].fillna('-1', inplace=True) #Se aplica -1 a los crimenes que no aplican
#Se establece NaN a aquellos valores que deberán recalcularse
df_crime['edad'] = df_crime['edad'].apply(lambda data: np.NaN if data == 'NO REPORTADA' else data )
df_crime['edad'] = df_crime['edad'].apply(lambda data: np.NaN if data == 'NOREPORTADO' else data )
df_crime['edad'] = df_crime['edad'].apply(lambda data: np.NaN if data == 'NO REPORTADO' else data )
df_crime['edad'] = df_crime['edad'].apply(lambda data: np.NaN if data == '-' else data )
df_crime['edad'] = df_crime['edad'].apply(lambda data: np.NaN if data == '-' else data )

df_crime['edad'] = pd.to_numeric(df_crime['edad'], downcast='integer')

#PENDIENTE REALIZAR LA INTERPOLACION PARA HALLAR LOS VALORES DE LOS NULOS (Los que aparecian como información NO REPORTADA)

In [81]:
#'HURTO COMERCIO', 'HURTO A ENTIDADES COMERCIALES','HURTO A ENTIDADES FINANCIERAS', 'TERRORISMO'
df_crime['escolaridad'].fillna('NO APLICA', inplace=True) 
df_crime['escolaridad'] = df_crime['escolaridad'].apply(lambda data: 'NO REPORTADO' if data == '-' else data )
df_crime['escolaridad'] = df_crime['escolaridad'].apply(lambda data: 'NO REPORTADO' if data == 'NO REPORTADA' else data )
df_crime['escolaridad'] = df_crime['escolaridad'].apply(lambda data: 'NO REPORTADO' if data == 'NOREPORTADO' else data )

In [82]:
df_crime['estadocivil'].fillna('NO APLICA', inplace=True)  #'HURTO COMERCIO', 'HURTO A ENTIDADES COMERCIALES','HURTO A ENTIDADES FINANCIERAS', 'TERRORISMO'
df_crime['estadocivil'] = df_crime['estadocivil'].apply(lambda data: 'NO REPORTADO' if data == '-' else data )
df_crime['estadocivil'] = df_crime['estadocivil'].apply(lambda data: 'NO REPORTADO' if data == 'NO REPORTADA' else data )
df_crime['estadocivil'] = df_crime['estadocivil'].apply(lambda data: 'NO REPORTADO' if data == 'NO REPORTA' else data )
df_crime['estadocivil'] = df_crime['estadocivil'].apply(lambda data: 'NO REPORTADO' if data == 'NOREPORTADO' else data )

In [83]:
def convert_to_time (field): 
    if(isinstance(field, datetime.time)):
        result = field
    else:
        result = datetime.datetime.strptime(time.strftime('%H:%M:%S', time.gmtime(float(field) * 3600 )), '%H:%M:%S').time()
    return result

df_crime['hora'] = df_crime.apply(lambda row: convert_to_time(row['hora']), axis=1)

In [95]:
df_crime['fecha_hora'] = df_crime.apply(lambda row: datetime.datetime.combine(row['fecha'], row['hora']), axis=1)

In [84]:
df_crime['movilagresor'] = df_crime['movilagresor'].apply(lambda data: 'NO REPORTADO' if data == '-' else data )
df_crime['movilagresor'] = df_crime['movilagresor'].apply(lambda data: 'NO REPORTADO' if data == 'NOREPORTADO' else data )

In [85]:
df_crime['movilvictima'] = df_crime['movilvictima'].apply(lambda data: 'NO REPORTADO' if data == '-' else data )
df_crime['movilvictima'] = df_crime['movilvictima'].apply(lambda data: 'NO REPORTADO' if data == 'NOREPORTADO' else data )

In [86]:
df_crime['paisnacimiento'].fillna('NO APLICA', inplace=True) 
df_crime['paisnacimiento'] = df_crime['paisnacimiento'].apply(lambda data: 'NO REPORTADO' if data == '-' else data )
df_crime['paisnacimiento'] = df_crime['paisnacimiento'].apply(lambda data: 'NO REPORTADO' if data == 'NOREPORTADO' else data )
df_crime['paisnacimiento'] = df_crime['paisnacimiento'].apply(lambda data: 'NO REPORTADO' if data == 'NO REPORTA' else data )

In [87]:
#'HURTO COMERCIO' 'HURTO A ENTIDADES COMERCIALES'  'HURTO A ENTIDADES FINANCIERAS' 
#'HOMICIDIOS EN ACCIDENTES DE TRANSITO' 'TERRORISMO'
df_crime['profesion'].fillna('NO APLICA', inplace=True) 
df_crime['profesion'] = df_crime['profesion'].apply(lambda data: 'NO REPORTADO' if data == '-' else data )
df_crime['profesion'] = df_crime['profesion'].apply(lambda data: 'NO REPORTADO' if data == 'NOREPORTADO' else data )
df_crime['profesion'] = df_crime['profesion'].apply(lambda data: 'NO REPORTADO' if data == 'NO REPORTADA' else data )

In [56]:
dict_crimen = {'HOMICIDIOS ACCIDENTES TRANSITO': 'HOMICIDIOS EN ACCIDENTES DE TRANSITO',
               'HUTO DE MOTOCICLETAS': 'HURTO DE MOTOCICLETAS',
               'LESIONES EN ACCIDENTES DE TRANSITO': 'LESIONES ACCIDENTES TRANSITO'}

df_crime['crimen'] = df_crime['crimen'].apply(lambda st: st.replace('-', ' ')\
                                                           .replace('2010', '')\
                                                           .replace('2011', '')\
                                                           .replace('2012', '')\
                                                           .replace('2019', '')\
                                                           .replace('_', '')\
                                                           .strip())

df_crime['crimen'] = df_crime['crimen'].apply(lambda data: dict_crimen[data] if data in dict_crimen.keys() else data)

#Creating dictionary of crime types
df_crime_type = pd.DataFrame(df_crime['crimen'].drop_duplicates().reset_index(drop=True))
df_crime_type['crime_type_id'] = df_crime_type.index
dct_crime_type = {}
for index, row in df_crime_type.iterrows():
    dct_crime_type[row['crimen']] = index

#Saving data to database
df_crime_type.to_sql(name='crime_type', con=engine, if_exists = 'append', index=False)

In [113]:
ds_barrio_convencion = os.path.join(ds_path, 'processed\\neighborhood_convention.csv') 
df_barrio_convencion = pd.read_csv(ds_barrio_convencion)
df_barrio_convencion['barrio_convencion_id'] = df_barrio_convencion.index
df_barrio_convencion.to_sql(name='barrio_convencion', con=engine, if_exists = 'append', index=False)

In [120]:
dct_barrio = {}
for index, row in df_barrio_convencion.iterrows():
    dct_barrio[row['nombre_barrio']] = row['barrio_id']

In [65]:
df_crime['crime_type_id'] = df_crime['crimen'].apply(lambda data: dct_crime_type[data])

In [ ]:
def search_neighborhood(neighborhood, dct):
    neighborhood_id = np.NaN
    if (neighborhood in dct):
        neighborhood_id = dct[neighborhood]
    return neighborhood_id

df_crime['barrio_id'] = df_crime['barrio'].apply(lambda data: search_neighborhood(data, dct_barrio))
df_crime = df_crime[~df_crime['barrio_id'].isna()]

In [168]:
df_crime.reset_index(drop=True, inplace=True)
df_crime['crime_id'] = df_crime.index

In [169]:
df_crime[df_crime['crime_id']==0]

,armaempleada,cantidad,clase,claseempleado,claseempresa,clasesitio,color,delito,edad,escolaridad,...,movilagresor,movilvictima,paisnacimiento,profesion,sexo,zona,crime_type_id,fecha_hora,barrio_id,crime_id
0,ARMA DE FUEGO,1,NO APLICA,INDEPENDIENTE,NO APLICA,TROCHA,NO APLICA,NO APLICA,46,SECUNDARIA,...,A PIE,A PIE,COLOMBIA,NO REPORTADO,FEMENINO,URBANA,0,2012-10-04 03:00:00,97,0


In [170]:
df_crime.to_sql(name='crime', con=engine, if_exists = 'append', index=False)

In [98]:
#Delete unnecessary columns from the dataframe
del df_crime['crimen']
del df_crime['codigodane']
del df_crime['municipio']
del df_crime['q']
del df_crime['dia']
del df_crime['fecha']
del df_crime['hora']
del df_crime['departamento']
del df_crime['barrio']

In [171]:
df_crime.isna().any()

armaempleada      False
cantidad          False
clase             False
claseempleado     False
claseempresa      False
clasesitio        False
color             False
delito            False
edad              False
escolaridad       False
estadocivil       False
linea             False
marca             False
modelo            False
movilagresor      False
movilvictima      False
paisnacimiento    False
profesion         False
sexo              False
zona              False
crime_type_id     False
fecha_hora        False
barrio_id         False
crime_id          False
dtype: bool